In [1]:
import pandas as pd
import numpy as np
import pymysql
import datetime as dt
import yfinance as yf
import re

pd.set_option("display.max_rows", 100)

In [2]:
#### TIMING THE SCRIPT FOR RUNNING:
run_start = dt.datetime.now()
run_start

datetime.datetime(2024, 2, 25, 23, 55, 15, 662858)

In [3]:
### calendario anbima:

file = '/home/peon/Documents/feriados_nacionais.xls'
df_holly = pd.read_excel(file)
df_holly = df_holly[~df_holly.Feriado.isnull()]
df_holly.loc[:, 'Data'] = df_holly.Data.apply(lambda x: x.date())



#### SQL QUERIES

In [4]:
# POSTGRES CONNECTION;

import psycopg2

conn = psycopg2.connect(dbname="carteira", user="peon")

cursor = conn.cursor()




In [5]:
# 1. load the raw data from B3, treat and clean and then load into the main table B3;

def get_cols_name(table_name):

    query = """SELECT *
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = N'""" + table_name + "'"
    
    cursor.execute(query)

    return pd.DataFrame(cursor.fetchall()).sort_values(4).iloc[:, 3].values


def query_df(query, columns):

    cursor.execute(query)
    df = cursor.fetchall()
    df = pd.DataFrame(df, columns = columns)

    return df


df_b3_raw = query_df("SELECT * FROM extrato_b3_raw",
        ['vector', 'date', 'event', 'asset', 'broker', 'quantidade', 'pu', 'amount']
)

df_b3_raw = df_b3_raw.astype(str).applymap(lambda x: x.strip())  # a must

df_b3_raw.head(2)

,vector,date,event,asset,broker,quantidade,pu,amount
0,Debito,31/05/2023,Transferência - Liquidação,KEPL3 - KEPLER WEBER S/A,CLEAR CORRETORA - GRUPO XP,200.00,$8.76,"$1,752.00"
1,Credito,31/01/2023,Transferência - Liquidação,BBDC4 - BANCO BRADESCO S/A,CLEAR CORRETORA - GRUPO XP,100.00,$13.71,"$1,371.00"


In [6]:
print(df_b3_raw.dtypes) # wich dataypes do we have

df_b3_raw.isna().sum() # NA Values on the dataset?

vector        object
date          object
event         object
asset         object
broker        object
quantidade    object
pu            object
amount        object
dtype: object


vector        0
date          0
event         0
asset         0
broker        0
quantidade    0
pu            0
amount        0
dtype: int64

In [7]:
# first --> treat date type.
df_b3 = df_b3_raw.copy()

df_b3.loc[:, 'date'] = df_b3_raw.date.apply(lambda x: dt.datetime.strptime(x, '%d/%m/%Y').strftime("%Y-%m-%d")) # we're going to discard the time data, keeping only date.
df_b3 = df_b3.sort_values('date')
#done

In [8]:
# diving into event;
df_b3.groupby('event').first()

# podemos concluir que, para nossa análise, faz sentido coletar Dividendo, Transferência - Liquidação, Juros Sobre Capital Próprio, Compra, Venda
# vamos remapear esses valores com dicionario para melhor serem buscados;
remap_event_vals = dict(zip(['Transferência - Liquidação', 'Venda', 'Compra'], ['Swingt'] * 3))
remap_event_vals['Juros Sobre Capital Próprio'] = 'JCP'

df_b3.replace({'event': remap_event_vals}, inplace=True)

In [9]:
# Dive into Asset
print(df_b3.asset)

# here, we need to regex the ticker code, getting only derivatives and stocks and throwing away Treasure bonds;

p = re.compile('[A-Z]{4,5}[0-9]{1,3}')

df_b3.loc[:, 'asset'] = df_b3.asset.apply(lambda x: p.search(x).group() if p.search(x) else 'drop')

df_b3 = df_b3[df_b3.asset != 'drop'] # removing anyone who didn't matched the regex above.

118                                   Tesouro Selic 2027
117                                   Tesouro IPCA+ 2045
65                 SANB4 - BANCO SANTANDER (BRASIL) S.A.
63            CXSE3 - CAIXA SEGURIDADE PARTICIPACOES S/A
64                                    KLBN4 - KLABIN S/A
                             ...                        
225                    Opção de Compra - MGLUB140 - MGLU
224    TAEE4 - TRANSMISSORA ALIANCA DE ENERGIA ELETRI...
223                     Opção de Venda - CSNAN192 - CSNA
222                     Opção de Venda - CSNAN192 - CSNA
221                    Opção de Compra - RRRPB260 - RRRP
Name: asset, Length: 316, dtype: object


In [10]:
# numeric types

df_b3[['pu', 'quantidade']] = df_b3[['pu', 'quantidade']].applymap(
    lambda x: float(x.replace(',', '').replace('$', ''))
)

### Prepare a .sql file for injection!

In [11]:
target_cols = ['vector', 'date', 'event', 'asset', 'quantidade', 'pu']  # columns that match or database design.

vals = []

for i, row in df_b3[target_cols].iterrows():
    vals.append(str(tuple(row.values)))

string = ",\n".join(vals)

inje_f = '/home/peon/Downloads/injection.sql'

with open(inje_f, 'w') as f:
    f.write("INSERT INTO extrato_b3 (vector, date, event, asset, quantidade, pu) VALUES\n" + string)




### At this point, we need to first load the generated file above to continue.

In [12]:
import sys

keep_going = input("The injection file was consumed?")

if keep_going == 'yes':
    None

else: 
    #sys.exit
    None


In [13]:
#  now, we get at a 2nd pillar of the script. It's time to create and update the wallet;
# this will be done making updates for every single row added in the extrato. So at every new row, update the database and then query for it again.
# first - check if there is a wallet done. if not, create one. the first row must be a credit op. we could lock the database to not allow negative quantities. That would garantee that we only sell if we have the position.
# another way is to check it here with assert.
# In a day, we can alter the database for updating the position. The next day,  we start with a copy of the day's before position.



In [14]:
df_ops = query_df("SELECT * FROM extrato_b3 WHERE event = 'Swingt'", get_cols_name('extrato_b3')) # df_ops == DataFrame of the operations in our extrato excluded JPC and Dividends;
df_ops = df_ops.applymap(lambda x: x.strip() if isinstance(x, str) else x) # maldito strip bugado.
# as we're going to only update the transactions, let's exclude dividends and jcp.

df_ops.head(2)

# lets correct qtd for debit. 
debcred_factor_correction = [1 if x=='Credito' else -1 for x in df_ops.vector  ]

df_ops['quantidade'] = df_ops.quantidade * debcred_factor_correction

In [15]:
def clone_position(dayp1, day):

    cursor.execute("INSERT INTO carteira (date, ticker, quantidade, pumedio) (SELECT '{}', ticker, quantidade, pumedio FROM carteira WHERE date = '{}')".format(dayp1, day))    

    return print("position cloned!")


def update_position(newqt, newpu, id):
            
        cursor.execute("""UPDATE carteira SET quantidade = {}, pumedio = {} WHERE id = {}""".format(newqt, "%.2f" % newpu, id))
        
        return print("Position updated!")



# main block for transactions and to update the position/custody
delta_time = (df_ops.date.max() - df_ops.date.min())
range_time = [df_ops.date.max() - dt.timedelta(days = x) for x in range(delta_time.days + 1)] # this create our date range object.
period = range_time.copy()
period.sort()



def fx_nxt_dia_util(day):

    dayp1 = day.date() + dt.timedelta(days = 1) # starts getting next day.

    while (dayp1.weekday() == 5 or dayp1.weekday() == 6 or 
    dayp1 in df_holly.Data.values):

        dayp1 = day.date() + dt.timedelta(days = 1) 

    return dayp1
    
    

In [19]:
stop here.

SyntaxError: invalid syntax (1551605323.py, line 1)

### UPDATING THE WALLET old old old

#### CALL CELL

In [ ]:
### ADD COTAÇÃO NA CARTEIRAS ATRAVÉS DO DICIONÁRIO; - FIZ SEPARADO PARA NÃO TER QUE MODIFICAR O MAIN;

wallet_wprices = {}
# GET MARKET PRICES

def get_MarketPrice(dia, df):
    
    dia = dt.datetime.strptime(dia, '%Y%m%d')
    diaMais  = dia + dt.timedelta(5)
    
    #date as string;
    dia_str = dia.strftime('%Y-%m-%d')
    diaMais_str = diaMais.strftime('%Y-%m-%d')
    
    #---
    tickers = df.index
    tickers = [x+'.SA' for x in tickers]

    df_prices = yf.download(tickers, start = dia_str, end = diaMais_str).iloc[[0]].Close.T
    df_prices.applymap(lambda x: round(x, 2))
    df_prices.columns = ['PUMercado']
    df_prices.index = [x[:-3] for x in df_prices.index] 
    
    #join and calc:   
    
    df = df.join(df_prices)
    df['ValorMercado'] = round(df.PUMercado * df.Qt, 2)
    df['Return (%)'] = round((df.PUMercado/ df.PU)*100 - 100, 2)
    
    #solving NAN PROBLEM with ENBR;
    df = df.fillna(0)
    
    return df


# chamada da função.
for dia, dataframe in wallet_dic.items():
    
    df = get_MarketPrice(dia, dataframe)
    wallet_wprices[dia] = df

    

[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  16 of 16 completed
[*********************100%%**********************]  17 of 17 completed
[*********************100%%**********************]  17 of 17 completed
[*********************100%%**********************]  17 of 17 completed
[*********************100%%**********************]  17 of 17 completed
[*****